ハッシュタグを目的変数に、ツイートの分類器を作る。今回はVtuberを題材にしました。  
一部不適切な内容のツイートを拾ってしまっています。申し訳ございません。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


スクレイピングのコードはこちら→https://github.com/Kiro02/TweetClasifier/blob/master/searcher.py

In [120]:
df_1 = pd.read_csv("キズナアイ.csv")
df_2 = pd.read_csv("輝夜月.csv")
df_3 = pd.read_csv("バーチャルのじゃロリ狐娘Youtuberおじさん.csv")
df_4 = pd.read_csv("ミライアカリ.csv")
df_5 = pd.read_csv("月ノ美兎.csv")

In [121]:
df = pd.concat([df_1,df_2,df_3,df_4,df_5])
df = df.reset_index()

In [122]:
del df["Unnamed: 0"] #よくわからない列ができてしまったので消す

In [123]:
df.head()

,index,hashtag,text
0,0,キズナアイ,#キズナアイ\nこれは前にアニメイト行った時に買ったやつ✨✨一目惚れ(♥ω♥*)ｷｭﾝｷｭﾝ...
1,1,キズナアイ,遂にやっちゃったキズナアイ面接っっ😆✨すっごいテンポ早いんだね！！！とろちゃんはついていける...
2,2,キズナアイ,#キズナアイ\nこの前ガチャガチャやった時のやつ！\n本当にかわいい！💕 https://t...
3,3,キズナアイ,#丸勘パジャマでどこまで行けるか\n#丸勘うちわ\n#のんのんびより\n#のんのんびよりギャ...
4,4,キズナアイ,「両極端は、あい交わる」 #キズナアイ


In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 3 columns):
index      434 non-null int64
hashtag    434 non-null object
text       434 non-null object
dtypes: int64(1), object(2)
memory usage: 10.2+ KB


・ツイートは最大１４０文字なのでそれを固定長にする（CNNなので可変長はNG)  
・ハッシュタグそのものは取り除く  
・データは生放送とかでワードに偏りが出てしまうので、いろいろな時期から撮りたい  
→サーバー上で定期的に動かすようにする  
・改行文字などを消去する  
・URLを除外  

In [127]:
#前処理はMeCabで。
import MeCab
m = MeCab.Tagger ("-Ochasen")
#テスト
print(m.parse(df.loc[2,["text"]][0]))

#	#	#	名詞-サ変接続		
キズナアイ	キズナアイ	キズナアイ	名詞-一般		
この	コノ	この	連体詞		
前	マエ	前	名詞-副詞可能		
ガチャガチャ	ガチャガチャ	ガチャガチャ	名詞-一般		
やっ	ヤッ	やる	動詞-自立	五段・ラ行	連用タ接続
た	タ	た	助動詞	特殊・タ	基本形
時	トキ	時	名詞-非自立-副詞可能		
の	ノ	の	助詞-連体化		
やつ	ヤツ	やつ	名詞-代名詞-一般		
！	！	！	記号-一般		
本当に	ホントウニ	本当に	副詞-一般		
かわいい	カワイイ	かわいい	形容詞-自立	形容詞・イ段	基本形
！	！	！	記号-一般		
💕	💕	💕	記号-一般		
https	https	https	名詞-固有名詞-組織		
://	://	://	名詞-サ変接続		
t	t	t	名詞-一般		
.	.	.	名詞-サ変接続		
co	co	co	名詞-一般		
/	/	/	名詞-サ変接続		
cmUnsIXl	cmUnsIXl	cmUnsIXl	名詞-一般		
1	1	1	名詞-数		
o	o	o	名詞-固有名詞-組織		
EOS



In [128]:
string="#キズナアイ\nこの前ガチャガチャやった時のやつ！\n本当にかわいい！"
string.replace("\n","")
string.strip()
print(string)

#キズナアイ
この前ガチャガチャやった時のやつ！
本当にかわいい！


In [129]:
test = [ord(x) for x in string.strip()] #文章をベクトル化
print(test)
print(len(string),len(test))

[35, 12461, 12474, 12490, 12450, 12452, 10, 12371, 12398, 21069, 12460, 12481, 12515, 12460, 12481, 12515, 12420, 12387, 12383, 26178, 12398, 12420, 12388, 65281, 10, 26412, 24403, 12395, 12363, 12431, 12356, 12356, 65281]
33 33


In [130]:
def replace_words(dataframe, word1, word2):
    """
    ツイート内のword1をword2置換
    """
    for i in range(len(dataframe.loc[:,["text"]])):
        dataframe.loc[i,["text"]] = dataframe.loc[i,["text"]].astype(str)[0].replace(str(word1),str(word2))
    return dataframe

In [131]:
df = replace_words(df, "\n", "") #改行記号削除
#ハッシュタグ削除
for vtuber in ["キズナアイ","輝夜月","ミライアカリ","バーチャルのじゃロリ狐娘Youtuberおじさん","月ノ美兎"]:
    df = replace_words(df, ("#" + vtuber), "")
df.tail()

,index,hashtag,text
429,95,月ノ美兎,#すーぱーそに子 まさかのコラボhttps://t.co/Mpw3WZsigw
430,96,月ノ美兎,すき https://t.co/WjyXWgOhzM
431,97,月ノ美兎,みとみとが真面目に話してるところに上から100トンがポンポン落ちてきてるの面白すぎる
432,98,月ノ美兎,月ノ美兎の放課後ラジオ #6 配信を開始しました #委員長すき #にじさんじ https:...
433,99,月ノ美兎,公式ﾗｲﾊﾞｰ情報まとめ9月9日更新https://t.co/WasJr6KzCJ#鈴谷アキ...


In [132]:
#ツイートの固定長ベクトル化
def string2vec(dataframe):
    array = np.arange(140).reshape(1,140)
    for i in range(len(dataframe.loc[:,["text"]])): #ベクトル化
        string_vec = [ord(x) for x in dataframe.loc[i,["text"]].astype(str)[0].strip()]
        while len(string_vec) < 140:     #140の長さで固定するためのゼロパディング
            string_vec.append(0)
        string_array = np.array(string_vec)
        string_array = string_array.reshape(1,140)
        array = np.append(array, string_array, axis=0)
    A = pd.DataFrame(array).iloc[1:,:].reset_index()
    dataframe = pd.concat([dataframe,A], axis=1)
    return dataframe

In [118]:
string2vec(df)

,index,hashtag,text,index,0,1,2,3,4,5,...,130,131,132,133,134,135,136,137,138,139
0,0,キズナアイ,これは前にアニメイト行った時に買ったやつ✨✨一目惚れ(♥ω♥*)ｷｭﾝｷｭﾝ♡*゜ http...,1,12371,12428,12399,21069,12395,12450,...,0,0,0,0,0,0,0,0,0,0
1,1,キズナアイ,遂にやっちゃったキズナアイ面接っっ😆✨すっごいテンポ早いんだね！！！とろちゃんはついていける...,2,36930,12395,12420,12387,12385,12419,...,85,75,100,0,0,0,0,0,0,0
2,2,キズナアイ,この前ガチャガチャやった時のやつ！本当にかわいい！💕 https://t.co/cmUnsI...,3,12371,12398,21069,12460,12481,12515,...,0,0,0,0,0,0,0,0,0,0
3,3,キズナアイ,#丸勘パジャマでどこまで行けるか#丸勘うちわ#のんのんびより#のんのんびよりギャラリー#劇場...,4,35,20024,21208,12497,12472,12515,...,0,0,0,0,0,0,0,0,0,0
4,4,キズナアイ,「両極端は、あい交わる」,5,12300,20001,26997,31471,12399,12289,...,0,0,0,0,0,0,0,0,0,0
5,5,キズナアイ,生放送！視聴者の代表とエッチなことして繋がりたい！#相互 #エロ #AV #エロ動画#コスプ...,6,29983,25918,36865,65281,35222,32884,...,0,0,0,0,0,0,0,0,0,0
6,6,キズナアイ,今日は休憩時間は来れなかったので仕事終わりにキズナアイコラボカフェ☺️ドリンクにデザインされ...,7,20170,26085,12399,20241,25001,26178,...,51,100,85,70,119,121,71,0,0,0
7,7,キズナアイ,https://t.co/PSbdxTq0Ji,8,104,116,116,112,115,58,...,0,0,0,0,0,0,0,0,0,0
8,8,キズナアイ,アイちゃんきた！(o^^o)めっちゃ嬉しいw＃オセロニア ＃キズナアイ https://t....,9,12450,12452,12385,12419,12435,12365,...,0,0,0,0,0,0,0,0,0,0
9,9,キズナアイ,大歓喜（2500円） https://t.co/S6PqgXMyO1,10,22823,27475,21916,65288,50,53,...,0,0,0,0,0,0,0,0,0,0


ツイートをベクトルにすることができた。次はCNNにかけていく。